In [1]:
%matplotlib inline

Both simulations are the summary taken from these 2 articles:
* https://www.statisticshowto.com/probability-and-statistics/t-test/
* https://www.statisticshowto.com/probability-and-statistics/hypothesis-testing/f-test/

Data for our simulation is taken from MysIDC (Malaysia Informative Data Centre). Kudos for Malaysia Government because now they have take a small initiative in reducing data illiteracy among Malaysian citizen.

# T-test

T-test is a statistical tool that is used by analysts to help comparing things, specifically comparing the mean difference between two things of a same type. For instance, let's say a pet company is selling cat foods Brand A and Brand B. At the end of year, the company's stakeholders ask the CEO to present data for manufacturing cost of the company. A good CEO will not just telling the stakeholders if they have increase/decrease manufacturing cost, but also must enlighten the stakeholders on what is the cause of the increase/decrease of the manufacturing cost. Maybe the manufacturing cost is driven by the cat food Brand A, and not by Brand B. Or maybe it is the other way around? 

So in order to find out the answer to the question above, what the CEO can do is to use the T-test to find out if it is true that there is a significant different between Brand A manufacturing cost and Brand B manufacturing cost.

There are 3 things that T-test is useful for:
* Compare means between two groups (Independent Sample T-test)
* Compare means of a group but with 2 different times (Paired Sample T-test)
* Compare one mean from the population mean (One Sample T-test)

For the simulation below, I will go through briefly how the T-test is used and will use the Crustacean & Molluscs data from MysIDC as an example.

In [28]:
import pandas as pd
import numpy as np

path = 'C:/Users/farea/Desktop/10.3.7 Production of Crustacean & Molluscs (Brackishwater Aquaculture) Malaysia 2000 - 2019_dataset.xlsx'

data = pd.read_excel(path)
data

,Category,Sub-Category,Year,Quantity ('000 Tonnes),Value (RM Million)
0,\nCrustacean & Molluscs,Cockle,2000,64.400,50.650
1,\nCrustacean & Molluscs,Cockle,2001,70.820,55.730
2,\nCrustacean & Molluscs,Cockle,2002,78.710,62.400
3,\nCrustacean & Molluscs,Cockle,2003,71.070,56.770
4,\nCrustacean & Molluscs,Cockle,2004,64.560,54.210
...,...,...,...,...,...
95,\nCrustacean & Molluscs,Shrimp,2015,58.000,1221.550
96,\nCrustacean & Molluscs,Shrimp,2016,43.253,918.632
97,\nCrustacean & Molluscs,Shrimp,2017,45.783,1067.337
98,\nCrustacean & Molluscs,Shrimp,2018,45.920,1114.950


In [8]:
data['Sub-Category'].unique()

array(['Cockle', 'Mussel', 'Oyster', 'Seaweed', 'Shrimp'], dtype=object)

Since T-test can only be used to compare two things, I will choose seaweed and shrimp because usually I will put these two into the dishes that I cook. I would like to know if there is a different in the quantity being produce in Malaysia so that when I go to the grocery, I will know which seafood should I buy (since usually if we have abundant source of food, the price would go low).

In [34]:
data = data[(data['Sub-Category']=='Shrimp') | (data['Sub-Category']=='Seaweed')]
data['Sub-Category'].unique()

array(['Seaweed', 'Shrimp'], dtype=object)